# Data format overview
## In this tutorial, we will overview evaluate the following data formats
1. avro (structured)
2. csv (semi-structured)
3. json (semi-structured)
4. orc (structured)
5. parquet (structured) 

## We evaluate the data formats via:
1. Disk usage
2. Read/Write latency
3. Random data lookup
4. Filtering/GroupBy(column-wise)
5. Distinct(row-wise)

In [31]:
import os
import s3fs
endpoint = "https://"+os.environ['AWS_S3_ENDPOINT']
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': endpoint})
event_log_path="pengfei/spark-history"

fs.touch('s3://'+event_log_path+'/.keep')
fs.info('pengfei/pengfei_test')

{'Key': 'pengfei/pengfei_test',
 'name': 'pengfei/pengfei_test',
 'type': 'directory',
 'Size': 0,
 'size': 0,
 'StorageClass': 'DIRECTORY'}

In [32]:
from pyspark.sql import SparkSession

spark = SparkSession \
       .builder.master("k8s://https://kubernetes.default.svc:443") \
       .appName("Evaluate data format") \
       .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") \
       .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
       .config("spark.executor.instances", "5") \
       .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
       .config("spark.eventLog.enabled","true") \
       .config("spark.eventLog.dir","s3a://"+event_log_path) \
       .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.0.1") \
       .getOrCreate()


In [33]:
json_data_path="s3a://pengfei/sspcloud-demo/data_format/netflix.json"
parquet_data_path="s3a://pengfei/sspcloud-demo/data_format/netflix.parquet"
avro_data_path="s3a://pengfei/sspcloud-demo/data_format/netflix.avro"
orc_data_path="s3a://pengfei/sspcloud-demo/data_format/netflix.orc"
csv_data_path="s3a://pengfei/sspcloud-demo/data_format/netflix.csv"

# Some useful functions for evaluating data format

### The read function read the source data file and convert it to a spark data frame

In [1]:
# file path for storing operation stats
data_format_op_stats_path="../tmp/op-stats.csv"

# file path for storing size stats
data_format_size_stats_path="../tmp/size-stats.csv"
def write_stats(line):
    file1 = open(data_format_op_stats_path,"a")
    file1.write(line+"\n")

In [46]:
import time
def read(fmt):
    start = time.time()
    if fmt == "json":
        sdf = spark.read.option("header", "true").json(json_data_path)
    elif fmt == "csv":
        sdf = spark.read.option("header", "true").csv(csv_data_path)
    elif fmt == "avro":
        sdf = spark.read.format("avro").load(avro_data_path)
    elif fmt == "parquet":
        sdf = spark.read.parquet(parquet_data_path)
    elif fmt == "orc":
        sdf = spark.read.orc(orc_data_path)
    sdf.show(5,False)
    stats="{}, {}, {}".format(fmt, "read", time.time() - start)
    write_stats(stats)
    print(stats)
    return sdf

### The get_shape function prints the shape(e.g. row number and column number) of the data frame

In [37]:
def get_shape(df,fmt):
    start = time.time()
    row_num=df.count()
    col_num=len(df.columns)
    stats="{}, {}, {}".format(fmt, "get_shape", time.time() - start)
    write_stats(stats)
    print("The data frame has {} rows and {} columns".format(row_num,col_num))
    print(stats)

### The stats function prints the min, max and numbers of a column of the data frame

In [38]:
def stats(df,fmt, field="rating"):
    start = time.time()
    max=df.agg({field: "max"})
    min=df.agg({field: "min"})
    count=df.agg({field: "count"})
    min.show(5,False)
    max.show(5,False)
    count.show(5,False)
    stats="{}, {}, {}".format(fmt, "stats", time.time() - start)
    write_stats(stats)
    print(stats)

### The random_batch function randomly select rows from the data frame. It can evaluate the ability of random data lookup

In [39]:
def random_batch(df,fmt):
    start = time.time()
    result=df.sample(False, 0.05).collect()
    stats="{}, {}, {}".format(fmt, "random_batch", time.time() - start)
    write_stats(stats)
    print(stats)
   # return result

### The distinct function count distinct rows of the data frame

In [40]:
def distinct(df,fmt):
    start = time.time()
    result = df.distinct().count()
    stats="{}, {}, {}".format(fmt, "distinct", time.time() - start)
    write_stats(stats)
    print(stats)
    return result

### The group_by function group and count the data frame by a specific column

In [41]:
def group_by(df,fmt):
    start = time.time()
    result=df.groupBy("rating").count()
    result.show(5,False)
    stats="{}, {}, {}".format(fmt, "group_by", time.time() - start)
    write_stats(stats)
    print(stats)
    #return result

### The filtering function filter data by using a specific boolean condition

In [42]:
def filtering(df, fmt, date="2005-11-15"):
    start = time.time()
    result = df.filter(df.date > date).count()
    stats="{}, {}, {}".format(fmt, "filtering", time.time() - start)
    write_stats(stats)
    print(stats)
    return result

### The remove_space function removes space in string of a column to avoid filtering fail.

In [ ]:
from pyspark.sql.functions import ltrim,rtrim,trim
def remove_space(df,col_name,position):
    # remove left side space
    if position =="l":
        return df.withColumn("tmp",ltrim(col(col_name))).drop(col_name).withColumnRenamed("tmp", col_name)
    # remove right side space
    elif position =="r":
        return df.withColumn("tmp",rtrim(col(col_name))).drop(col_name).withColumnRenamed("tmp", col_name)
    # remove all side space
    elif position =="a":
        return df.withColumn("tmp",trim(col(col_name))).drop(col_name).withColumnRenamed("tmp", col_name)

In [43]:
def saveCSV(df,outputPath,fileName): Unit ={
    df.coalesce(1).write.mode("overwrite")
      .option("header","true")
      .option("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")
      .option("encoding", "UTF-8")
      .option("delimiter", ",") 
      .csv(outputPath+"/"+fileName)
  }   

# 1. Gathering stats of each data format

## 1.1 Get CSV format evaluation stats

In [44]:
csv_df=read("csv")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|1488844|3     |2005-09-06|
|822109 |5     |2005-05-13|
|885013 |4     |2005-10-19|
|30878  |4     |2005-12-26|
|823519 |3     |2004-05-03|
+-------+------+----------+
only showing top 5 rows

csv, read, 8.758424282073975


In [13]:
csv_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- date: string (nullable = true)



In [15]:
get_shape(csv_df,"csv")

The data frame has 24058262 rows and 3 columns
csv, get_shape, 6.950009822845459


In [16]:
# get min, max and row number of column rating
stats(csv_df,"csv",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

csv, stats, 22.299869537353516


In [17]:
random_batch(csv_df,"csv")

csv, random_batch, 15.357799291610718


In [18]:
distinct(csv_df,"csv")

csv, distinct, 19.20894479751587


12168704

In [19]:
group_by(csv_df,"csv")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

csv, group_by, 8.280380249023438


In [20]:
filtering(csv_df,"csv")

In [28]:
name="netflix"
csv_df.write.mode("overwrite").option("header", "true").csv("{}.csv".format(name))

## 1.2 Get Json format evaluation stats

In [98]:
json_df=read("json")

+----------+------+-------+
|date      |rating|user_id|
+----------+------+-------+
|2005-09-06|3     |1488844|
|2005-05-13|5     |822109 |
|2005-10-19|4     |885013 |
|2005-12-26|4     |30878  |
|2004-05-03|3     |823519 |
+----------+------+-------+
only showing top 5 rows

json, read, 9.966949701309204


In [99]:
json_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- user_id: string (nullable = true)



In [100]:
get_shape(json_df,"json")

In [111]:
# get min, max and row number of column rating
stats(json_df,"json",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

json, stats, 35.458019971847534


In [101]:
random_batch(json_df,"json")

json, random_batch, 17.13827896118164


In [102]:
distinct(json_df,"json")

json, distinct, 21.003305673599243


12168704

In [115]:
group_by(json_df,"json")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

json, group_by, 12.402800798416138


DataFrame[rating: string, count: bigint]

In [103]:
filtering(json_df,"json")

json, filtering, 11.020655632019043


850269

## 1.3 Get Avro format evaluation stats

In [47]:
avro_df=read("avro")

Py4JJavaError: An error occurred while calling o196.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 9) (10.233.118.242 executor 5): java.lang.ClassCastException: cannot assign instance of java.lang.invoke.SerializedLambda to field org.apache.spark.sql.execution.datasources.FileFormat$$anon$1.dataReader$1 of type scala.Function1 in instance of org.apache.spark.sql.execution.datasources.FileFormat$$anon$1
	at java.io.ObjectStreamClass$FieldReflector.setObjFieldValues(ObjectStreamClass.java:2301)
	at java.io.ObjectStreamClass.setObjFieldValues(ObjectStreamClass.java:1431)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2411)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2329)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2187)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1667)
	a

In [ ]:
get_shape(avro_df,"avro")

In [ ]:
stats(avro_df,"avro",field="rating")

In [ ]:
random_batch(avro_df,"avro")

In [ ]:
distinct(avro_df,"avro")

In [ ]:
group_by(avro_df,"avro")

In [ ]:
filtering(avro_df,"avro")

## 1.4 Get Parquet format evaluation stats

In [141]:
parquet_df=read("parquet")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|1488844|3     |2005-09-06|
|822109 |5     |2005-05-13|
|885013 |4     |2005-10-19|
|30878  |4     |2005-12-26|
|823519 |3     |2004-05-03|
+-------+------+----------+
only showing top 5 rows

parquet, read, 1.6400139331817627


In [105]:
get_shape(parquet_df,"parquet")

The data frame has 24058262 rows and 3 columns
parquet, get_shape, 1.010782241821289


In [106]:
random_batch(parquet_df,"parquet")

parquet, random_batch, 6.159161567687988


In [112]:
stats(parquet_df,"parquet",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

parquet, stats, 4.589794635772705


In [137]:
distinct(parquet_df,"parquet")

parquet, distinct, 164.0264663696289


12168704

In [117]:
group_by(parquet_df,"parquet")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

parquet, group_by, 1.5566697120666504


In [119]:
filtering(parquet_df,"parquet")

parquet, filtering, 1.3744680881500244


850269

# 1.5 Get ORC format evaluation stats

In [130]:
orc_df=read("orc")

+-------+------+----------+
|user_id|rating|date      |
+-------+------+----------+
|1488844|3     |2005-09-06|
|822109 |5     |2005-05-13|
|885013 |4     |2005-10-19|
|30878  |4     |2005-12-26|
|823519 |3     |2004-05-03|
+-------+------+----------+
only showing top 5 rows

orc, read, 2.3085367679595947


In [132]:
get_shape(orc_df,"orc")

The data frame has 24058262 rows and 3 columns
orc, get_shape, 1.4532253742218018


In [133]:
random_batch(orc_df,"orc")

orc, random_batch, 6.4762678146362305


In [134]:
stats(orc_df,"orc",field="rating")

+-----------+
|min(rating)|
+-----------+
|1          |
+-----------+

+-----------+
|max(rating)|
+-----------+
|5          |
+-----------+

+-------------+
|count(rating)|
+-------------+
|24053764     |
+-------------+

orc, stats, 4.612210035324097


In [138]:
distinct(orc_df,"orc")

orc, distinct, 185.58755350112915


12168704

In [135]:
group_by(orc_df,"orc")

+------+-------+
|rating|count  |
+------+-------+
|3     |6904181|
|null  |4498   |
|5     |5506583|
|1     |1118186|
|4     |8085741|
+------+-------+
only showing top 5 rows

orc, group_by, 1.680478811264038


In [136]:
filtering(orc_df,"orc")

orc, filtering, 1.186652421951294


850269

# 2. Compare the stats of different format 

## 2.1 Disk Usage for different  file format

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, DoubleType,StringType,LongType

In [ ]:
# read size stats file

rawSizeDf=spark.read.option("header", "true").csv(data_format_size_stats_path,schema=size_schema)
# generate a new column to make data humain readable
sizeDf=rawSizeDf.withColumn("size_mb",col("size_kb")/1024)
sizeDf.orderBy(col("size_kb").asc()).show(5)

In [ ]:
# convert spark df to pandas df
pd_size_df=sizeDf.toPandas()

In [ ]:
# show the stats in a catplot
g_size = sns.catplot(
    data=pd_size_df, kind="bar",
    x="format", y="size_mb", 
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=8
)
g_size.despine(left=False)
g_size.set_axis_labels("", "Size (kb) ")
g_size.set(ylim=(150, 1300))

The main objective of any file is to store data. It is necessary to store large volumes of different data types with less spaces. Because the disk usage can cost you a lot of money.

Based on the above graphe, we can have the obvious conclusion **do not use JSON or csv to store the raw data, and orc, parquet are the best solution for storing data.Now we want to know why?** 

Note that we used the default configuration for all format during the entire benchmark. So we do not specify any compression codec anywhere. As a result, we have the following table

| format | size(mb) | compression |
|--------|----------|-------------|
| json   | 1261.90  | None        |
| csv    | 472.10   | None        |
| avro   | 279.91   | None        |
| parquet| 198.01   | SNAPPY      |
| orc    | 186.84   | ZLIB        |


1. JSON uses the most space, because it has huge overhead on storing "schema". Each row must repeat the column name value, which are the same for all the rows (in our example, it represents 24058262 rows). 

2. CSV only has one line header for storing column names, so it uses much less space than JSON  
3. Avro does not do any compression, but it still uses less space than CSV. Because it saves raw data in binary. And the binary codec do a bit compression with storing int and long leveraging variable-length zig-zag coding.
4. Parque alson stores raw data in binary, and it does compress data. In spark, it uses SNAPPY as the default compressing codec.
5. Orc uses ZLIB as compression codec. As ZLIB can offer better compression ratio, so Orc use less space than Parquet. But Snappy is faster. 


You could argu that, you can gzip CSV and json which will reduce the disk usage. But, json and csv are not splitable after compression. As a result, we can not benefit from the parallel processing of Spark anymore. Thus, the advantage of avro, parquet and orc is obvious.

Note that avro, parquet and orc supports many compression codec. Here we only benchmark the default setting of each format. So the difference of disk usage between these three are negligible.

## 2.2 Visualize the data processing latency for each format

In this section, we will examine the latency of several the most common data processing operations to determine which format is more optimal in certain context. We will first examine the common operations one by one. At last, we will give you an overview and conclusion

In [ ]:
# define a schema

op_schema = StructType([
    StructField("format", StringType(), True),
    StructField("command", StringType(), True),
    StructField("Latency", DoubleType(), True)])

size_schema = StructType([
    StructField("format", StringType(), True),
    StructField("size_kb", LongType(), True)])


# read stats file
rawDf=spark.read.option("header", "false").csv(data_format_op_stats_path,schema=op_schema)
rawDf.show(5)
rawDf.count()

In [ ]:
# remove duplicated rows
tmpDf=rawDf.dropDuplicates(["format","command"])
# remove space in command column
statsDf=remove_space(tmpDf,"command","l")
statsDf.count()

In [ ]:
# convert to pandas df
pd_df=statsDf.toPandas()
pd_df.shape

### 2.2.1 Read data

In [2]:
read_op_df=pd_df[pd_df.command.eq("read")]
read_op_df.head(5)

NameError: name 'pd_df' is not defined

In [ ]:
# show the stats in a catplot
g_read_op = sns.catplot(
    data=read_op_df, kind="bar",
    x="format", y="Latency", 
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=8
)
g_read_op.despine(left=False)
g_read_op.set_axis_labels("", "Latency (seconds) ")
g_read_op.set(ylim=(0, 10))

You could notice that the reading speed of orc, parquet and avro are much faster than csv and json. Because they store raw data in binary, which are optimized for performence.

### 2.2.2 Get basic stats such as min, max, column numbers

The basic stats of a data set such as min, max, row numbers are the basic information we will gather when we exploring un data set. Below figure shows the latency of each format.

In [ ]:
meta_op_df=pd_df.loc[(pd_df.command.eq("stats"))|(pd_df.command.eq("get_shape"))]
meta_op_df.head(5)

In [ ]:
g_op_meta = sns.catplot(
    data=meta_op_df, kind="bar",
    x="format", y="Latency", hue="command",
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=10
)
g_op_meta.despine(left=False)
g_op_meta.set_axis_labels("", "Latency (seconds)")
g_op_meta.legend.set_title(" ")

Orc and Parquet saves the min, max and row numbers as metadata. So the basic stats operations do not require the process of the entire dataset.  

### 2.2.3 Random batch

Random batch select randomly a subset of rows in a dataframe. If checks the performence of radom access operations of a data format. Unlike sequential access, random access requires more sophisticated optimization to avoid read unnecessary data. 

In [ ]:
random_op_df=pd_df[pd_df.command.eq("random_batch")]
random_op_df.head(5)

In [ ]:
# show the stats in a catplot
g_random_op = sns.catplot(
    data=random_op_df, kind="bar",
    x="format", y="Latency", 
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=8
)
g_random_op.despine(left=False)
g_random_op.set_axis_labels("", "Latency (seconds) ")
g_random_op.set(ylim=(0, 10))

orc, parquet, and avro split data in small chunks, and each chunk has a header which contains the metadata of the data in this chunk. As a result, when a certain row is required, the reader will check first the metadata in the header, if the condition is not matched, this chunk will be omitted. And this will avoid to read all unnecessary data 

### 2.2.4 column-wise operations (Filtering/GroupBy)

Many operations such as filtering or groupBy only instrested in data of certain columns. If we can avoid reading unnecessary data. We can improve dramatiquely the performence. Below figure shows the latency of filtering and groupBy operations. 

In [ ]:
col_op_df=pd_df.loc[(pd_df.command.eq("group_by"))|(pd_df.command.eq("filtering"))]
col_op_df.head(5)

In [ ]:
g_col_op = sns.catplot(
    data=col_op_df, kind="bar",hue="command",
    x="format", y="Latency", 
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=8
)
g_col_op.despine(left=False)
g_col_op.set_axis_labels("", "Latency (seconds) ")
g_col_op.set(ylim=(0, 10))

We can notice orc and parquet are much quicker than avro, csv, and json. Because orc, and parquet are columnar-based data formats, and avro, csv, and json are row-based data formats. We will discuss the difference between columnar-based and row-based in another tutorial. 

### 2.2.5.row-wise operation (Distinct)

In [ ]:
distinct_op_df=pd_df[pd_df.command.eq("distinct")]
distinct_op_df.head(5)

In [ ]:
g_distinct_op = sns.catplot(
    data=distinct_op_df, kind="bar",
    x="format", y="Latency", 
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=8
)
g_distinct_op.despine(left=False)
g_distinct_op.set_axis_labels("", "Latency (seconds) ")
g_distinct_op.set(ylim=(0, 50))

### 2.2.6 Overview

In [ ]:
g_op = sns.catplot(
    data=pd_df, kind="bar",
    x="format", y="Latency", hue="command",
    order=['orc','parquet','avro','csv','json'],
    ci="sd", palette="dark", alpha=0.8, height=10
)
g_op.despine(left=False)
g_op.set_axis_labels("", "Latency (seconds)")
g_op.legend.set_title(" ")


# Stop the spark cluster

In [29]:
# stop sparksession
spark.sparkContext.stop()

### Check if the spark cluster is well closed. You should not see any python-spark pods

In [34]:
! kubectl get pods

NAME                                                     READY   STATUS      RESTARTS   AGE
deleting-pods-with-completed-status-1615824000-2nwxn     0/1     Completed   0          58m
jupyter-1615800186-567f67779b-cbmgx                      1/1     Running     0          7h35m
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-1   1/1     Running     0          21s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-2   1/1     Running     0          21s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-3   1/1     Running     0          20s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-4   1/1     Running     0          20s
python-spark-sql-basic-example-6e3a6b7836d2d444-exec-5   1/1     Running     0          20s
ubuntu-1612965548-79c9567b44-nhs9p                       1/1     Running     1          33d
